In [ ]:
pip install pandas langchain faiss-cpu

In [ ]:
#--------------------PART 2 : CREATE AND SAVE THE VECTOR STORE FOR THE IMF EXCEL FILE--------------------------
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from langchain.docstore.document import Document
import pandas as pd

# Load Excel file
df = pd.read_excel("/content/WEO.xlsx")

# Create documents where the page_content is the JSON representation of each row
excel_docs = [Document(page_content=row.dropna().to_json()) for _, row in df.iterrows()]

# Create the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Extract text content to batch-embed
texts = [doc.page_content for doc in excel_docs]

# Batch-embed all texts at once
embeddings = embedding_model.embed_documents(texts)

# Combine texts and embeddings into a list of tuples
text_embedding_pairs = list(zip(texts, embeddings))

# Create the FAISS vectorstore
vectorstore = FAISS.from_embeddings(text_embedding_pairs, embedding_model)

# Now your embeddings are stored in the 'vectorstore' object.
# You can save it for later use:
vectorstore.save_local("/content/excel_vectorstore")

print("FAISS vectorstore created and saved successfully!")